# Table of Contents
 <p><div class="lev1 toc-item"><a href="#reading-and-preparing-data" data-toc-modified-id="reading-and-preparing-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>reading and preparing data</a></div><div class="lev1 toc-item"><a href="#Trajector-Clustering-on-Dimension-reduction" data-toc-modified-id="Trajector-Clustering-on-Dimension-reduction-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Trajector Clustering on Dimension reduction</a></div><div class="lev1 toc-item"><a href="#Trajectory-Clustering-by-Weight" data-toc-modified-id="Trajectory-Clustering-by-Weight-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Trajectory Clustering by Weight</a></div>

# reading and preparing data

In [4]:
#library(ggbiplot)
library(tidyverse, quiet=T)
library(traj)
library(caret, quiet=T)

In [9]:


dat0 <- read.csv("../training_ultrasound.csv")

# Use Hadlok equation to estimate fetal weight from the 4 key ultrasound measurements
dat.raw = dat0 %>%
    mutate(
        LOG10.FWT.GM = 1.3596 + 0.0064*HCIRCM + 0.0424*ABCIRCM + 0.174*FEMURCM + 0.00061*BPDCM*ABCIRCM - 0.00386*ABCIRCM*FEMURCM,
        WTKG.estimate = ifelse(AGEDAYS<1 ,(10^LOG10.FWT.GM)/1000 ,WTKG) ,
        Study = paste('Study', STUDYID)
        )
dat.raw %>% glimpse
# clean data, remove samples with only one observation and after-birth
dat = dat.raw %>% group_by(SUBJID) %>%
    mutate(tot.measurements = n()) %>%
    filter(tot.measurements >1) %>%
    filter(AGEDAYS<0)


dat.train.raw = dat %>% ungroup %>%
    select(GAGEDAYS, SUBJID, ABCIRCM, HCIRCM, BPDCM, FEMURCM, WTKG.estimate) %>%
    filter(complete.cases(.)) %>% group_by(SUBJID) %>%
    mutate(measurementId = row_number(), tot_measurements=n()) %>%
#    filter(measurementId < 7) %>%
#    filter(tot_measurements >1) %>% 
    ungroup
#     filter(GAGEDAYS > 150, GAGEDAYS <210)

dat.train = dat.train.raw %>% dplyr::select(-WTKG.estimate)

dat.train.raw %>% head

Observations: 17,370
Variables: 37
$ STUDYID       <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...
$ SUBJID        <int> 1002, 1002, 1002, 1003, 1003, 1003, 1003, 1003, 1003,...
$ SEXN          <int> 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...
$ SEX           <fctr> Female, Female, Female, Male, Male, Male, Male, Male...
$ GAGEBRTH      <int> 276, 276, 276, 280, 280, 280, 280, 280, 280, 280, 280...
$ BIRTHWT       <int> 3540, 3540, 3540, 3100, 3100, 3100, 3100, 3100, 3100,...
$ BIRTHLEN      <dbl> 50.3, 50.3, 50.3, 50.3, 50.3, 50.3, 50.3, 50.3, 50.3,...
$ BIRTHHC       <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N...
$ DELIVERY      <fctr> Category 2.0, Category 2.0, Category 2.0, Category 2...
$ PARITY        <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...
$ GRAVIDA       <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...
$ GAGEDAYS      <int> 255, 277, 669, 223, 224, 263, 266, 281, 321, 463, 645...
$ AGEDAYS       <

GAGEDAYS,SUBJID,ABCIRCM,HCIRCM,BPDCM,FEMURCM,WTKG.estimate,measurementId,tot_measurements
255,1002,32.5,33.4,9.1,7.3,3.0632504,1,1
223,1003,25.6,29.2,8.2,6.3,1.7124993,1,4
224,1003,25.6,29.2,8.2,6.3,1.7124993,2,4
263,1003,30.2,31.0,9.1,7.2,2.6278574,3,4
266,1003,30.2,31.0,9.1,7.2,2.6278574,4,4
169,1005,18.7,21.9,5.9,4.1,0.5989562,1,3


In [12]:
dat.lm = dat %>% glm ( BWT_40~ABCIRCM*GAGEDAYS+HCIRCM*GAGEDAYS+BPDCM*GAGEDAYS+FEMURCM*GAGEDAYS, data=.) 
dat.lm %>% summary
par(mfrow=c(2,2))
#dat.lm %>% plot
dat %>% select(BWT_40) %>% head
predict(dat.lm, dat) %>% head


Call:
glm(formula = BWT_40 ~ ABCIRCM * GAGEDAYS + HCIRCM * GAGEDAYS + 
    BPDCM * GAGEDAYS + FEMURCM * GAGEDAYS, data = .)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.12286  -0.20024  -0.01422   0.18330   2.58416  

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       4.657e+00  8.519e-02  54.666  < 2e-16 ***
ABCIRCM          -4.573e-02  1.711e-02  -2.673  0.00753 ** 
GAGEDAYS         -2.775e-02  7.033e-04 -39.460  < 2e-16 ***
HCIRCM            2.903e-01  2.732e-02  10.625  < 2e-16 ***
BPDCM            -5.056e-01  8.221e-02  -6.150 8.12e-10 ***
FEMURCM           1.173e-01  7.728e-02   1.517  0.12925    
ABCIRCM:GAGEDAYS  7.091e-04  7.217e-05   9.826  < 2e-16 ***
GAGEDAYS:HCIRCM  -1.268e-03  1.176e-04 -10.785  < 2e-16 ***
GAGEDAYS:BPDCM    2.731e-03  3.672e-04   7.437 1.14e-13 ***
GAGEDAYS:FEMURCM  5.897e-05  3.366e-04   0.175  0.86096    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispers

Adding missing grouping variables: `SUBJID`


SUBJID,BWT_40
1002,3.614882
1003,3.100000
1003,3.100000
1003,3.100000
1003,3.100000
1005,2.988224


1        2        3        4        5        6 
3.565806 3.232805 3.208931 3.158320 3.097166 3.277695

In [13]:
dat.lm = dat %>% glm ( BWT_40~ABCIRCM+HCIRCM+BPDCM+FEMURCM, weight=GAGEDAYS, data=.) 
dat.lm %>% summary
par(mfrow=c(2,2))
#dat.lm %>% plot
dat %>% select(BWT_40) %>% head
predict(dat.lm, dat) %>% head


Call:
glm(formula = BWT_40 ~ ABCIRCM + HCIRCM + BPDCM + FEMURCM, data = ., 
    weights = GAGEDAYS)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-18.672   -3.350   -0.196    3.176   37.746  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.259438   0.026496 123.018  < 2e-16 ***
ABCIRCM      0.062979   0.003104  20.291  < 2e-16 ***
HCIRCM      -0.019525   0.005644  -3.459 0.000544 ***
BPDCM       -0.000543   0.017079  -0.032 0.974639    
FEMURCM     -0.188605   0.016132 -11.691  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 25.68918)

    Null deviance: 216208  on 7927  degrees of freedom
Residual deviance: 203535  on 7923  degrees of freedom
  (148 observations deleted due to missingness)
AIC: 6093.1

Number of Fisher Scoring iterations: 2


Adding missing grouping variables: `SUBJID`


SUBJID,BWT_40
1002,3.614882
1003,3.100000
1003,3.100000
1003,3.100000
1003,3.100000
1005,2.988224


1        2        3        4        5        6 
3.272376 3.108918 3.108918 3.193244 3.193244 3.233072

In [22]:
dat.lm = dat %>% glm ( BWT_40~ABCIRCM+HCIRCM+BPDCM+FEMURCM, weight=GAGEDAYS, data=.) 
dat.lm %>% summary
par(mfrow=c(2,2))
#dat.lm %>% plot

dat.predict = dat %>% select(GAGEDAYS, BWT_40) 
dat.predict$estimate = predict(dat.lm, dat)
dat.predict %>% head
dat.predict %>% gather(var, value, -SUBJID, -GAGEDAYS ) %>% ggplot(aes(x))


Call:
glm(formula = BWT_40 ~ ABCIRCM + HCIRCM + BPDCM + FEMURCM, data = ., 
    weights = GAGEDAYS)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-18.672   -3.350   -0.196    3.176   37.746  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.259438   0.026496 123.018  < 2e-16 ***
ABCIRCM      0.062979   0.003104  20.291  < 2e-16 ***
HCIRCM      -0.019525   0.005644  -3.459 0.000544 ***
BPDCM       -0.000543   0.017079  -0.032 0.974639    
FEMURCM     -0.188605   0.016132 -11.691  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 25.68918)

    Null deviance: 216208  on 7927  degrees of freedom
Residual deviance: 203535  on 7923  degrees of freedom
  (148 observations deleted due to missingness)
AIC: 6093.1

Number of Fisher Scoring iterations: 2


Adding missing grouping variables: `SUBJID`


SUBJID,GAGEDAYS,BWT_40,estimate
1002,255,3.614882,3.272376
1003,223,3.100000,3.108918
1003,224,3.100000,3.108918
1003,263,3.100000,3.193244
1003,266,3.100000,3.193244
1005,169,2.988224,3.233072


SUBJID,GAGEDAYS,var,value
1002,255,BWT_40,3.614882
1003,223,BWT_40,3.100000
1003,224,BWT_40,3.100000
1003,263,BWT_40,3.100000
1003,266,BWT_40,3.100000
1005,169,BWT_40,2.988224
